# Find duplicate questions on StackOverflow by their embeddings

In this assignment you will learn how to calculate a similarity for pieces of text. Using this approach you will know how to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

In this task you will you will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [1]:
! python3 -c "import ssl; print(ssl.OPENSSL_VERSION)"


OpenSSL 1.0.2g  1 Mar 2016


### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [2]:
from grader import Grader

In [3]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. You need to download it by following this [link](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing).
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [4]:
import gensim
from gensim import models


In [5]:
wv_embeddings = models.KeyedVectors.load_word2vec_format("data/GoogleNews-vectors-negative300.bin.gz", binary=True, limit=500000)

### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [6]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [7]:
print(check_embeddings(wv_embeddings))

These embeddings look good.


In [8]:
"1" in wv_embeddings

True

## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above. This function should work with the input text as is without any preprocessing.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [9]:
import numpy as np

In [66]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    vec = np.zeros((dim,), dtype=np.float32)
    count = 0
    for w in question.split():
        if w in embeddings:
            count += 1
            vec += embeddings[w]
    if count == 0:
        return vec
    return vec/count

To check the basic correctness of your implementation, run the function *question_to_vec_tests*.

In [11]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [12]:
print(question_to_vec_tests())

Basic tests are passed.


You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [15]:
import nltk
nltk.download("stopwords")
from util import array_to_string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
import pandas as pd
question = pd.read_csv("data/test_embeddings.tsv", sep = "/t")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [17]:
question.head()

,Play Framework 1.2.5 : OutOfMemoryError occured : Java heap space in play framework
0,Navigating between several views in iphone
1,A strange phenomenon of vector member in a class
2,Formula to auto change all calendar hours if I...
3,Is there a way to redirect the browser from th...
4,How to read response from angular resource $sa...


In [18]:
question2vec_result = []
for question in open('data/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

Current answer for task Question2Vec is: 0.019293891266
-0.0287272129208
0.0460561104119
0.0852593332529
0.0243055559695
-0.0729031041265
0.0...


Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [19]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for current ranking
    """
    count = 0
    for rank in dup_ranks:
        if rank <=k:
            count += 1
    return count/(len(dup_ranks)+1e-8)

Test your code on the tiny examples:

In [20]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, 1]
    
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [21]:
print(test_hits())

Basic test are passed.


In [22]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for current ranking
    """
    count = 0
    for rank in dup_ranks:
        if k>=rank:
            count += 1/np.log2(1+rank)
    return count/(len(dup_ranks)+1e-8)

In [23]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [24]:
print(test_dcg())

Basic test are passed.


Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [25]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [26]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

Current answer for task HitsCount is: 0.9999999900000002
0.4999999975
0.999999995
0.4999999975
0.999999995
0.3333333322222222
0.6666666644...


In [27]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

Current answer for task DCGScore is: 0.99999999
0.4999999975
0.815464872708
0.4999999975
0.815464872708
0.333333332222
0.543643249378
0.7...


##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should upload *validation* corpus to evaluate current solution.

In [28]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [29]:
validation = read_corpus("data/validation.tsv")

In [79]:
prepared_validation[0]

['print binary heap tree without recursion',
 'best convert recursive function iterative one',
 'use ngmodel directive angular js',
 'flash drawing erasing',
 'toggle react component using hide show classname',
 'use usercontrol another project current webpage',
 'paths resolved differently upgrading aspnet 4',
 'materialize datepicker rendering icon clicked',
 'creating pypi package could find version satisfies requirement iso8601',
 'analyze confusion matrix',
 'declare c array swift',
 'using rand flipping coin rolling die',
 'handling json field special character name java',
 'react native select row listview push',
 'get creation_time video using ffmpeg regex',
 'row exist multiple',
 'specify classifier gradle dependencys dependency',
 'using unwind multiple documents',
 'visual studio code missing x509certificate2ui',
 'developing cordova plugins firefox os',
 'update multiple rows 2 columns mysql',
 'analyse format alphanumeric string',
 'dealing large tiff images c++ magick li

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [67]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    vecq = np.array([np.array(question_to_vec(question, embeddings, dim))])
    vecc = np.array([np.array(question_to_vec(can, embeddings, dim)) for can in candidates])
    scores = list(cosine_similarity(vecq, vecc)[0])
    tl = [(i, candidates[i], scores[i]) for i in range(len(candidates))]
    stl = sorted(tl, key=lambda x:x[2], reverse=True)
    return [(t[0],t[1]) for t in stl]

Test your code on the tiny examples:

In [33]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [34]:
print(test_rank_candidates())

Basic tests are passed.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [35]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [36]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.212 | Hits@   1: 0.212
DCG@   5: 0.267 | Hits@   5: 0.315
DCG@  10: 0.282 | Hits@  10: 0.363
DCG@ 100: 0.320 | Hits@ 100: 0.552
DCG@ 500: 0.353 | Hits@ 500: 0.811
DCG@1000: 0.373 | Hits@1000: 1.000


If you did all the steps correctly, you should be frustrated by the received results. Let's try to understand why the quality is so low. First of all, when you work with some data it is necessary to have an idea how the data looks like. Print several questions from the data:

As you can see, we deal with the raw data. It means that we have many punctuation marks, special characters and unlowercased letters. In our case, it could lead to the situation where we can't find some embeddings, e.g. for the word "grid?". 

To solve this problem you should use the functions *text_prepare* from the previous assignments to prepare the data.

In [37]:
from util import text_prepare

Now transform all the questions from the validation set:

In [38]:
prepared_validation = []
for line in validation:
    q, *ex = line
    q = text_prepare(q)
    for i, e in enumerate(ex):
        ex[i] = text_prepare(e)
    prepared_validation.append([q, *ex])

In [39]:
del validation

Let's evaluate the approach again after the preparation:

In [40]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

KeyboardInterrupt: 

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

Now, prepare also train and test data, because you will need it in the future:

In [ ]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [ ]:
prepare_file("data/train.tsv", "data/p_train_prepare.tsv")
prepare_file("data/test.tsv", "data/p_test.tsv")

**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [41]:
from util import matrix_to_string

In [42]:
w2v_ranks_results = []
prepared_test_data = "data/p_test.tsv"
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

Current answer for task W2VTokenizedRanks is: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...


## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace cannot be run on Windows and we recommend to use provided
[docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a number of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour.

In [ ]:
import pandas
import nltk

In [ ]:
!pwd

In [83]:
test = read_corpus("data/p_test.tsv")
test[1]

['phpseclib giving compatible server client encryption algorithms found',
 'insert multiple rows access database',
 'adding new files git branch shows file master branch well',
 'recursion print_r output mean',
 'automatically generated user id',
 'rounding showing cells',
 'css opaque overlay transparent div sections',
 'authentication internal external users',
 'add newlines mediawiki raw text via api',
 'notice compatible server client encryption algorithms found',
 'best practice empty belongs_to association',
 'object memory layout c++ structure depends platforms',
 'jquery pull text inside div p tag',
 'mapping multiple tables one list hibernate',
 'setting filenames tab',
 'alembic care migration files called',
 'choosing cloud storage service web api ftp third party server',
 'prototypejs selecting odd subset visible children',
 'query cpu utilization mysql running redhat',
 'model selfdependency onetomany field implementation',
 'zoom center image',
 'reliable scalable solutio

In [82]:
train[2]

['sending array via ajax fails', 'getting list items unordered list php']

In [50]:
!../Starspace/starspace train -trainFile 'data/p_train_prepare.tsv' -model "mdl" -trainMode 3 \
    -adagrad true -ngrams 1 -epoch 5 -dim 100 -minCount 2 -verbose False -fileFormat "labelDoc" \
    -negSearchLimit 10 -similarity "cosine" -lr 0.05

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
Start to initialize starspace model.
Build dict from input file : data/p_train_prepare.tsv
Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Loading data from file : data/p_train_prepare.tsv
Total number of examples loaded : 999740
Training epoch 0: 0.05 0.01
Epoch: 59.5%  lr: 0.044394  loss: 0.011283  eta: 0h16m  tot: 0h2m11s  (11.9%)lr: 0.049960  loss: 0.064660  eta: 0h18m  tot: 0h0m1s  (0.1%)1.6%  lr: 0.049920  loss: 0.046516  eta: 0h18m  tot: 0h0m3s  (0.3%)1.8%  lr: 0.049900  loss: 0.043926  eta: 0h18m  tot: 0h0m4s  (0.4%)2.2%  lr: 0.049850  loss: 0.040369  eta: 0h18m  tot: 0h0m5s  (0.4%)2.6%  lr

Epoch: 59.1%  lr: 0.034364  loss: 0.002699  eta: 0h11m  tot: 0h5m56s  (31.8%).0%  lr: 0.039920  loss: 0.001997  eta: 0h11m  tot: 0h3m56s  (20.2%)1.8%  lr: 0.039880  loss: 0.002376  eta: 0h11m  tot: 0h3m57s  (20.4%)2.2%  lr: 0.039830  loss: 0.002307  eta: 0h12m  tot: 0h3m58s  (20.4%)2.4%  lr: 0.039830  loss: 0.002239  eta: 0h11m  tot: 0h3m58s  (20.5%)2.8%  lr: 0.039800  loss: 0.002272  eta: 0h11m  tot: 0h3m59s  (20.6%)3.1%  lr: 0.039780  loss: 0.002339  eta: 0h11m  tot: 0h4m0s  (20.6%)5.0%  lr: 0.039540  loss: 0.002342  eta: 0h12m  tot: 0h4m3s  (21.0%)5.9%  lr: 0.039429  loss: 0.002400  eta: 0h12m  tot: 0h4m5s  (21.2%)6.0%  lr: 0.039409  loss: 0.002407  eta: 0h12m  tot: 0h4m5s  (21.2%)6.1%  lr: 0.039409  loss: 0.002407  eta: 0h12m  tot: 0h4m5s  (21.2%)%  lr: 0.039349  loss: 0.002380  eta: 0h12m  tot: 0h4m6s  (21.3%)7.2%  lr: 0.039289  loss: 0.002402  eta: 0h12m  tot: 0h4m7s  (21.4%)7.3%  lr: 0.039289  loss: 0.002411  eta: 0h12m  tot: 0h4m8s  (21.5%)8.5%  lr: 0.039199  loss: 0.002400  et

Epoch: 100.0%  lr: 0.030180  loss: 0.002673  eta: 0h10m  tot: 0h7m15s  (40.0%)9.8%  lr: 0.034284  loss: 0.002690  eta: 0h11m  tot: 0h5m58s  (32.0%)60.0%  lr: 0.034264  loss: 0.002696  eta: 0h11m  tot: 0h5m58s  (32.0%)60.1%  lr: 0.034244  loss: 0.002696  eta: 0h11m  tot: 0h5m58s  (32.0%)60.3%  lr: 0.034184  loss: 0.002698  eta: 0h11m  tot: 0h5m59s  (32.1%)60.6%  lr: 0.034184  loss: 0.002693  eta: 0h11m  tot: 0h5m59s  (32.1%)60.9%  lr: 0.034174  loss: 0.002691  eta: 0h11m  tot: 0h6m0s  (32.2%)62.0%  lr: 0.034004  loss: 0.002691  eta: 0h11m  tot: 0h6m2s  (32.4%)62.7%  lr: 0.033904  loss: 0.002697  eta: 0h11m  tot: 0h6m3s  (32.5%)62.8%  lr: 0.033884  loss: 0.002697  eta: 0h11m  tot: 0h6m4s  (32.6%)63.9%  lr: 0.033784  loss: 0.002683  eta: 0h11m  tot: 0h6m6s  (32.8%)64.0%  lr: 0.033784  loss: 0.002684  eta: 0h11m  tot: 0h6m6s  (32.8%)65.0%  lr: 0.033714  loss: 0.002696  eta: 0h11m  tot: 0h6m8s  (33.0%)65.2%  lr: 0.033714  loss: 0.002695  eta: 0h11m  tot: 0h6m8s  (33.0%)65.7%  lr: 0.033654  

Epoch: 57.8%  lr: 0.014174  loss: 0.001574  eta: 0h5m  tot: 0h12m31s  (71.6%)0.3%  lr: 0.019940  loss: 0.001507  eta: 0h7m  tot: 0h10m28s  (60.1%)1.2%  lr: 0.019820  loss: 0.001723  eta: 0h6m  tot: 0h10m30s  (60.2%)  lr: 0.019810  loss: 0.001773  eta: 0h6m  tot: 0h10m31s  (60.3%)1.5%  lr: 0.019790  loss: 0.001765  eta: 0h6m  tot: 0h10m31s  (60.3%)2.3%  lr: 0.019710  loss: 0.001640  eta: 0h6m  tot: 0h10m33s  (60.5%)2.4%  lr: 0.019680  loss: 0.001611  eta: 0h6m  tot: 0h10m33s  (60.5%)2.5%  lr: 0.019660  loss: 0.001614  eta: 0h6m  tot: 0h10m33s  (60.5%)3.1%  lr: 0.019600  loss: 0.001587  eta: 0h6m  tot: 0h10m34s  (60.6%)3.9%  lr: 0.019489  loss: 0.001520  eta: 0h6m  tot: 0h10m36s  (60.8%)4.1%  lr: 0.019449  loss: 0.001534  eta: 0h6m  tot: 0h10m36s  (60.8%)4.6%  lr: 0.019389  loss: 0.001589  eta: 0h6m  tot: 0h10m37s  (60.9%)4.8%  lr: 0.019369  loss: 0.001567  eta: 0h6m  tot: 0h10m38s  (61.0%)5.0%  lr: 0.019369  loss: 0.001545  eta: 0h6m  tot: 0h10m38s  (61.0%)6.2%  lr: 0.019269  loss: 0.00

Epoch: 100.0%  lr: 0.010140  loss: 0.001559  eta: 0h3m  tot: 0h14m19s  (80.0%)8.3%  lr: 0.014124  loss: 0.001573  eta: 0h5m  tot: 0h12m32s  (71.7%)58.8%  lr: 0.014074  loss: 0.001575  eta: 0h5m  tot: 0h12m33s  (71.8%)59.4%  lr: 0.013974  loss: 0.001582  eta: 0h5m  tot: 0h12m35s  (71.9%)59.7%  lr: 0.013954  loss: 0.001578  eta: 0h4m  tot: 0h12m35s  (71.9%)59.9%  lr: 0.013944  loss: 0.001578  eta: 0h4m  tot: 0h12m36s  (72.0%)60.0%  lr: 0.013944  loss: 0.001581  eta: 0h4m  tot: 0h12m36s  (72.0%)60.2%  lr: 0.013934  loss: 0.001580  eta: 0h4m  tot: 0h12m37s  (72.0%)60.7%  lr: 0.013924  loss: 0.001579  eta: 0h4m  tot: 0h12m37s  (72.1%)61.3%  lr: 0.013824  loss: 0.001580  eta: 0h4m  tot: 0h12m39s  (72.3%)61.8%  lr: 0.013784  loss: 0.001577  eta: 0h4m  tot: 0h12m40s  (72.4%)62.2%  lr: 0.013754  loss: 0.001577  eta: 0h4m  tot: 0h12m41s  (72.4%)62.6%  lr: 0.013714  loss: 0.001572  eta: 0h4m  tot: 0h12m42s  (72.5%)62.9%  lr: 0.013684  loss: 0.001569  eta: 0h4m  tot: 0h12m43s  (72.6%)63.3%  lr: 0.

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [51]:
import numpy as np
starspace_embeddings = {}
for line in open("./mdl.tsv"):
    word, *vec = line.strip().split()
    vf = []
    for v in vec:
        vf.append(float(v))
    starspace_embeddings[word] = np.array(vf)


In [73]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [74]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

DCG@   1: 0.520 | Hits@   1: 0.520
DCG@   5: 0.614 | Hits@   5: 0.696
DCG@  10: 0.634 | Hits@  10: 0.756
DCG@ 100: 0.666 | Hits@ 100: 0.910
DCG@ 500: 0.675 | Hits@ 500: 0.984
DCG@1000: 0.677 | Hits@1000: 1.000


Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [58]:
a = pd.read_csv("data/p_test.tsv")
a.head(1)

access structure member without knowing name\tgwt servlet classnotfoundexception\tslowcheetah transforming entityframework migrations\tdevelopment tools permission set\tgo scene without using button flash asc3\tobjectivec get last 2 characters string\tinserting rails link google maps infowindow\trequire files rails application\tdowhile loop r\txercesc++ dom node line column number location\tpython netcdf4 hdf5\tgenerating md5 hash qt\txsl scenario node present every loop\tabsolute beginners guide working audio c c++\tdescription item name paypal payment received email\tlatex annotation matlab\treplace slashes double quote characters using sed\tiostream library included codeblocks\tsafe processstart implementation untrusted url strings\ttrouble linking php database\tgoogle analytics api error\tpossible return one column using jpa\tneed htaccess redirection\tget full description activity cleartool\tcrystal reports cutting text pdf\tphantomjs mac os x works command line via exec\tdiff multiple files single base file\tremove script tags dom\tliferay login page redirect\twriting dataannotations best approach\tnet claim three namespaces\treturn value function color\tdatatable filter date range\tlayout sqlite resuts\tandroid calendar delete event\tsending email via gmail net 4\tvim regex searching alter placement cursor matched strings\twordpress rest api plugin get slow response\tbroken windowsandroid usbconnection device pc\tpointers primitive types c++\tangularfire impossible recover time entered user database\tc program crashes function called\tcleaner way right operator vector\tstream youtube video mediaelement windows store app\tcheck one vector subset another\tadd dictionarystring liststring datatable c#\tcheck whether object observablecollection\tdoesnt timer work\tspring data elastic search sort document types\tdetermine available physical memory linux\tcheck multiple values unique codeigniter\topensagres allowed use apache poilike packages\tuicolors values change serialization deserialization\tunable read checkbox gridview\tc++ java communication image processing application\tnothing showing installation field parsecom objc\tuser input restriction goes haywire\temail nodejs expressjs angularjs nodemiailer sending mail\tc# serialize class list elements type class\tfind mindistance parameter requestlocationupdates used\tcalling vbnet function javascript\tminimize function selection function call overhead\tdatagridview showing columns data\tdisable context menu internet explorer control\tbest way simple gameloop javascript\tsound app running iphone low\tusing phoenix cloudera hbase installed repo\tweb crawler working nested divs\timporting another module another subdirectory current directorys parent directory python\tcalculus engine vb\tipad problems table rows popovercontrol\tway make cell editable column readonly xtragrid\tpull uiscrollview hold position\twrite ironrouter nested template\traphael paper editing\tproblem jquery editinplace live function need ninja\tssrs using iff dividing\tapply lubridate\tcakephp 20 cannot save\tcuda injecting ptx function\texcel script hyperlink extraction\tsequential image presentation pygame\ttesting web application using angularjs mvc implementation\tpushing navigation controller supported ios 71 works fine ios 8\tsorting array nsstring + number using nssortdescriptor ios\tlarge photo version contacts android\tgce managed groups api horizontally scaling kubernetes nodes\tmethod sorting arabic words alphabetically ios7\tdelphi prism load winform without showing\tcant solve add androidsupportv4appfragment list applied makeapplicationfragment\tcontroller names restful routes optional\tplus 1 button doesnt render properly ie9\tmarkdown live preview textarea\tcreating new linear layout specific paramaters inside another linear layout\tinclude prebuilt static library existing aosp module\tcorrect way query hive spark maximum performance\tadding pagination repeater\tmultiple inputs using typeah

In [68]:
starspace_ranks_results = []
prepared_test_data = "data/p_test.tsv"
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

Current answer for task StarSpaceRanks is: 85	57	60	28	23	98	91	89	46	70	36	25	4	26	74	49	22	47	16	94	78	24	11	39	84	13	76	99	83	15	53	51	71	87...


### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [71]:
STUDENT_EMAIL = "15700080242@163.com" # EMAIL 
STUDENT_TOKEN = "OReZTRzdRlU9YX51"# TOKEN 
grader.status()

You want to submit these parts:
Task Question2Vec: 0.019293891266
-0.0287272129208
0.0460561104119
0.0852593332529
0.0243055559695
-0.0729031041265
0.0...
Task HitsCount: 0.9999999900000002
0.4999999975
0.999999995
0.4999999975
0.999999995
0.3333333322222222
0.6666666644...
Task DCGScore: 0.99999999
0.4999999975
0.815464872708
0.4999999975
0.815464872708
0.333333332222
0.543643249378
0.7...
Task W2VTokenizedRanks: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...
Task StarSpaceRanks: 85	57	60	28	23	98	91	89	46	70	36	25	4	26	74	49	22	47	16	94	78	24	11	39	84	13	76	99	83	15	53	51	71	87...


If you want to submit these answers, run cell below

In [72]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
